In [ ]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 34.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=cf90f970934730925eb63ae7d5c0b16fefc87f15b4535c040025ba64942cdc63
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder

In [ ]:
model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/My Drive/NLPWork/

/content/drive/My Drive/NLPWork


In [ ]:
import json
import numpy as np

In [ ]:
results = {}
def get_relevant_sentences(cs):
    global results

    for pmcid in cs:
        print(pmcid)
        if pmcid.startswith('0'):
            curr_ref_file = pmcid.split('_')[1]
        else:
            curr_ref_file = pmcid
        CURR_PMC = pmcid

        with open(f'Sentences/{curr_ref_file}.json') as f:
            sentences = json.load(f)

        # paras = text.split("\n\n")

        results[CURR_PMC] = {}
        idx = 0
        for file, d in cs[pmcid].items():
            if len(d['support']) == 0:
                continue
            claim = d['context']
            # print(f'Claim {idx}')
            idx += 1
            scores = model.predict([(claim, sentence) for sentence in sentences])
            sim_scores_argsort = np.argsort(scores)[::-1]

            current_sentences = []
            # Print out reranked results:
            for i in range(0, 10):
                # print(f'{i+1:2} {reranked[i].text}')
                current_sentences.append(sentences[sim_scores_argsort[i]])
            results[CURR_PMC][file] = current_sentences

In [ ]:
def calculate_recall(cs, results):
    def overlap(start, end, s, e):
        return start <= e and s <= end

    for pmcid in results:
        if pmcid.startswith('0'):
            curr_ref_file = pmcid.split('_')[1]
        else:
            curr_ref_file = pmcid
        with open(f'Articles/{curr_ref_file}.txt', encoding="utf-8") as f:
            content = f.read()
        for file in results[pmcid]:
            indices = [[content.find(s.strip()), content.find(s.strip())+len(s.strip())] for s in results[pmcid][file]]
            count = 0
            support_indices = [[content.find(s.strip()), content.find(s.strip())+len(s.strip())] for s in cs[pmcid][file]['support']]
            for start, end in support_indices:
                for s, e in indices:
                    if overlap(start, end, s, e):
                        count += 1
                        break
            cs[pmcid][file]['count'] = count
    res = 0
    total = 0
    for pmcid in cs:
        for file in cs[pmcid]:
            if 'count' in cs[pmcid][file]:
                res += (cs[pmcid][file]['count'] / len(cs[pmcid][file]['support']))
                total += 1

    return res/total

In [ ]:
with open('Eugene_ContextAndSupport.json') as f:
    cs1 = json.load(f)

with open('Eugene_ContextAndSupport_No_Title.json') as f:
    cs2 = json.load(f)

with open('Degaulle_ContextAndSupport.json') as f:
    cs3 = json.load(f)

with open('Degaulle_ContextAndSupport_No_Title.json') as f:
    cs4 = json.load(f)

In [ ]:
results = {}
get_relevant_sentences(cs1)
calculate_recall(cs1, results)

PMC2811155
PMC2848616
PMC3442244
PMC3522098
PMC3090745
PMC3650111
PMC3096954
PMC2958248
PMC3179858
PMC3607626


0.2942396313364055

In [ ]:
results = {}
get_relevant_sentences(cs2)
calculate_recall(cs2, results)

PMC2811155
PMC2848616
PMC3442244
PMC3522098
PMC3090745
PMC3650111
PMC3096954
PMC2958248
PMC3179858
PMC3607626


0.4774154589371981

In [ ]:
results = {}
get_relevant_sentences(cs3)
calculate_recall(cs3, results)

010_PMC3650111
004_PMC3090745
001_PMC2811155
007_PMC3442244
005_PMC3096954
006_PMC3179858
002_PMC2848616
003_PMC2958248
008_PMC3522098
009_PMC3607626


0.5973163841807909

In [ ]:
results = {}
get_relevant_sentences(cs4)
calculate_recall(cs4, results)

010_PMC3650111
004_PMC3090745
001_PMC2811155
007_PMC3442244
005_PMC3096954
006_PMC3179858
002_PMC2848616
003_PMC2958248
008_PMC3522098
009_PMC3607626


0.6293154761904762

In [ ]:
results = {}
def get_relevant_sentences_para(cs, num_para):
    global results

    for pmcid in cs:
        print(pmcid)
        if pmcid.startswith('0'):
            curr_ref_file = pmcid.split('_')[1]
        else:
            curr_ref_file = pmcid
        CURR_PMC = pmcid

        with open(f'Articles/{curr_ref_file}.txt') as f:
            text = f.read()

        paras = text.split("\n\n")
        paragraphs = []
        for p in paras:
            if "references" in p.lower():
                break
            paragraphs.append(p)

        results[CURR_PMC] = {}
        idx = 0
        for file, d in cs[pmcid].items():
            if len(d['support']) == 0:
                continue
            claim = d['context']
            # print(f'Claim {idx}')
            idx += 1
            scores = model.predict([(claim, p) for p in paragraphs])
            sim_scores_argsort = np.argsort(scores)[::-1]

            current_sentences = []
            # Print out reranked results:
            for i in range(0, num_para):
                # print(f'{i+1:2} {reranked[i].text}')
                current_sentences.append(paragraphs[sim_scores_argsort[i]])
            results[CURR_PMC][file] = current_sentences

In [ ]:
results = {}
get_relevant_sentences_para(cs1, 3)
calculate_recall(cs1, results)

PMC2811155
PMC2848616
PMC3442244
PMC3522098
PMC3090745
PMC3650111
PMC3096954
PMC2958248
PMC3179858
PMC3607626


0.29815668202764983

In [ ]:
results = {}
get_relevant_sentences_para(cs2, 3)
calculate_recall(cs2, results)

PMC2811155
PMC2848616
PMC3442244
PMC3522098
PMC3090745
PMC3650111
PMC3096954
PMC2958248
PMC3179858
PMC3607626


0.4714975845410629

In [ ]:
results = {}
get_relevant_sentences_para(cs3, 3)
calculate_recall(cs3, results)

010_PMC3650111
004_PMC3090745
001_PMC2811155
007_PMC3442244
005_PMC3096954
006_PMC3179858
002_PMC2848616
003_PMC2958248
008_PMC3522098
009_PMC3607626


0.5375706214689264

In [ ]:
results = {}
get_relevant_sentences_para(cs4, 3)
calculate_recall(cs4, results)

010_PMC3650111
004_PMC3090745
001_PMC2811155
007_PMC3442244
005_PMC3096954
006_PMC3179858
002_PMC2848616
003_PMC2958248
008_PMC3522098
009_PMC3607626


0.5663690476190475

In [ ]:
results = {}
get_relevant_sentences_para(cs1, 5)
calculate_recall(cs1, results)

PMC2811155
PMC2848616
PMC3442244
PMC3522098
PMC3090745
PMC3650111
PMC3096954
PMC2958248
PMC3179858
PMC3607626


0.40107526881720423

In [ ]:
results = {}
get_relevant_sentences_para(cs2, 5)
calculate_recall(cs2, results)

PMC2811155
PMC2848616
PMC3442244
PMC3522098
PMC3090745
PMC3650111
PMC3096954
PMC2958248
PMC3179858
PMC3607626


0.5884057971014492

In [ ]:
results = {}
get_relevant_sentences_para(cs3, 5)
calculate_recall(cs3, results)

010_PMC3650111
004_PMC3090745
001_PMC2811155
007_PMC3442244
005_PMC3096954
006_PMC3179858
002_PMC2848616
003_PMC2958248
008_PMC3522098
009_PMC3607626


0.5912429378531072

In [ ]:
results = {}
get_relevant_sentences_para(cs4, 5)

010_PMC3650111
004_PMC3090745
001_PMC2811155
007_PMC3442244
005_PMC3096954
006_PMC3179858
002_PMC2848616
003_PMC2958248
008_PMC3522098
009_PMC3607626


In [ ]:
calculate_recall(cs4, results)

0.6229166666666666